## One Hot Encoding of Frequent Categories

We learned in Section 4 that high cardinality and rare labels may result in certain categories appearing only in the train set, therefore causing over-fitting, or only in the test set, and then our models wouldn't know how to score those observations.

We also learned in the previous lecture on one hot encoding, that if categorical variables contain multiple labels, then by re-encoding them with dummy variables we will expand the feature space dramatically.

**In order to avoid these complications, we can create dummy variables only for the most frequent categories**

This procedure is also called one hot encoding of top categories.

In fact, in the winning solution of the KDD 2009 cup: ["Winning the KDD Cup Orange Challenge with Ensemble Selection"](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf), the authors limit one hot encoding to the 10 most frequent labels of the variable. This means that they would make one binary variable for each of the 10 most frequent labels only.

OHE of frequent or top categories is equivalent to grouping all the remaining categories under a new category. We will have a better look at grouping rare values into a new category in a later notebook in this section.


### Advantages of OHE of top categories

- Straightforward to implement
- Does not require hrs of variable exploration
- Does not expand massively the feature space
- Suitable for linear models


### Limitations

- Does not add any information that may make the variable more predictive
- Does not keep the information of the ignored labels


Often, categorical variables show a few dominating categories while the remaining labels add little information. Therefore, OHE of top categories is a simple and useful technique.

### Note

The number of top variables is set arbitrarily. In the KDD competition the authors selected 10, but it could have been 15 or 5 as well. This number can be chosen arbitrarily or derived from data exploration.


## In this demo:

We will see how to perform one hot encoding with: Feature-Engine using the House Prices dataset.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# for one hot encoding with feature-engine
from feature_engine.encoding import OneHotEncoder

In [2]:
# load dataset

data = pd.read_csv(
    "../../houseprice.csv",
    usecols=["Neighborhood", "Exterior1st", "Exterior2nd", "SalePrice"],
)

data.head()

Neighborhood Exterior1st Exterior2nd  SalePrice
0      CollgCr     VinylSd     VinylSd     208500
1      Veenker     MetalSd     MetalSd     181500
2      CollgCr     VinylSd     VinylSd     223500
3      Crawfor     Wd Sdng     Wd Shng     140000
4      NoRidge     VinylSd     VinylSd     250000

In [3]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data[["Neighborhood", "Exterior1st", "Exterior2nd"]],  # predictors
    data["SalePrice"],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0,
)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1022, 3), (438, 3))


## One hot encoding of top categories with Feature-Engine

### Advantages

- quick
- creates the same number of features in train and test set

### Limitations

- None to my knowledge

In [4]:
ohe_enc = OneHotEncoder(
    top_categories=10,  # you can change this value to select more or less variables
    # we can select which variables to encode
    variables=["Neighborhood", "Exterior1st", "Exterior2nd"],
    drop_last=False,
)

ohe_enc.fit(X_train)

OneHotEncoder(top_categories=10,
              variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

In [5]:
# in the encoder dict we can observe each of the top categories
# selected for each of the variables

ohe_enc.encoder_dict_

{'Neighborhood': ['NAmes',
  'CollgCr',
  'OldTown',
  'Edwards',
  'Sawyer',
  'Somerst',
  'Gilbert',
  'NWAmes',
  'NridgHt',
  'SawyerW'],
 'Exterior1st': ['VinylSd',
  'HdBoard',
  'Wd Sdng',
  'MetalSd',
  'Plywood',
  'CemntBd',
  'BrkFace',
  'WdShing',
  'Stucco',
  'AsbShng'],
 'Exterior2nd': ['VinylSd',
  'Wd Sdng',
  'HdBoard',
  'MetalSd',
  'Plywood',
  'CmentBd',
  'Wd Shng',
  'BrkFace',
  'AsbShng',
  'Stucco']}

In [6]:
# this is the list of variables that the encoder will transform

ohe_enc.variables_

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [7]:
X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

# let's explore the result
X_train.head()

Neighborhood_NAmes  Neighborhood_CollgCr  Neighborhood_OldTown  \
64                     0                     1                     0   
682                    0                     0                     0   
960                    0                     0                     0   
1384                   0                     0                     0   
1100                   0                     0                     0   

      Neighborhood_Edwards  Neighborhood_Sawyer  Neighborhood_Somerst  \
64                       0                    0                     0   
682                      0                    0                     0   
960                      0                    0                     0   
1384                     1                    0                     0   
1100                     0                    0                     0   

      Neighborhood_Gilbert  Neighborhood_NWAmes  Neighborhood_NridgHt  \
64                       0                    0                     0   
682                      0                    0                     0   
960                      0                    0                     0   
1384                     0                    0                     0   
1100                     0                    0                     0   

      Neighborhood_SawyerW  ...  Exterior2nd_VinylSd  Exterior2nd_Wd Sdng  \
64                       0  ...                    1                    0   
682                      0  ...                    0                    1   
960                      0  ...                    0                    0   
1384                     0  ...                    0                    0   
1100                     0  ...                    0                    1   

      Exterior2nd_HdBoard  Exterior2nd_MetalSd  Exterior2nd_Plywood  \
64                      0                    0                    0   
682                     0                    0                    0   
960                     0                    0                    1   
1384                    0                    0                    0   
1100                    0                    0                    0   

      Exterior2nd_CmentBd  Exterior2nd_Wd Shng  Exterior2nd_BrkFace  \
64                      0                    0                    0   
682                     0                    0                    0   
960                     0                    0                    0   
1384                    0                    1                    0   
1100                    0                    0                    0   

      Exterior2nd_AsbShng  Exterior2nd_Stucco  
64                      0                   0  
682                     0                   0  
960                     0                   0  
1384                    0                   0  
1100                    0                   0  

[5 rows x 30 columns]

**Note**

If the argument variables is left to None, then the encoder will automatically identify **all categorical variables**. Is that not sweet?

The encoder will not encode numerical variables. So if some of your numerical variables are in fact categories, you will need to re-cast them as object before using the encoder.